***GENERATED CODE FOR medicareprescriptiondrugspending PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import round
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'brand_name', 'transformation_label': 'String Indexer'}], 'feature': 'brand_name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'Abilify', 'max': 'Tracleer', 'missing': '0', 'distinct': '92'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'brand_name'}, {'feature_label': 'brand_name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('brand_name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'generic_name', 'transformation_label': 'String Indexer'}], 'feature': 'generic_name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Abatacept; Abatacept/Maltose', 'max': 'Treprostinil Sodium', 'missing': '0', 'distinct': '89'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'generic_name'}, {'feature_label': 'generic_name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('generic_name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'coverage_type', 'transformation_label': 'String Indexer'}], 'feature': 'coverage_type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Part B', 'max': 'Part D', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'coverage_type'}, {'feature_label': 'coverage_type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('coverage_type')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'total_spending', 'threshold': 448085386.374, 'transformation_label': 'Binarizer'}], 'feature': 'total_spending', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '500', 'mean': '469982921.3', 'stddev': '685975898.26', 'min': '0.0', 'max': '7.0306334858E9', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'total_spending'}, {'feature_label': 'total_spending', 'threshold': 448085386.374, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('total_spending')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'beneficiary_cost_share', 'threshold': 53223939.044, 'transformation_label': 'Binarizer'}], 'feature': 'beneficiary_cost_share', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '500', 'mean': '53223939.48', 'stddev': '76494193.61', 'min': '0.0', 'max': '3.9106178122E8', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'beneficiary_cost_share'}, {'feature_label': 'beneficiary_cost_share', 'threshold': 53223939.044, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('beneficiary_cost_share')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'total_annual_spending_per_user', 'threshold': 15359.996, 'transformation_label': 'Binarizer'}], 'feature': 'total_annual_spending_per_user', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '500', 'mean': '15360.47', 'stddev': '25215.13', 'min': '0.0', 'max': '162370.93', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'total_annual_spending_per...'}, {'feature_label': 'total_annual_spending_per_user', 'threshold': 15359.996, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop(
            'total_annual_spending_per_user')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'beneficiary_count_lis', 'transformation_label': 'String Indexer'}], 'feature': 'beneficiary_count_lis', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '254', 'mean': '128.85', 'stddev': '228.59', 'min': '0', 'max': '99,200', 'missing': '246', 'distinct': '238'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'beneficiary_count_lis'}, {'feature_label': 'beneficiary_count_lis', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('beneficiary_count_lis')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'beneficiary_count_no_lis', 'transformation_label': 'String Indexer'}], 'feature': 'beneficiary_count_no_lis', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '254', 'mean': '219.1', 'stddev': '296.05', 'min': '0', 'max': '971,937', 'missing': '246', 'distinct': '238'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'beneficiary_count_no_lis'}, {'feature_label': 'beneficiary_count_no_lis', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'beneficiary_count_no_lis')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'beneficiary_cost_share_lis', 'threshold': 3132957.16, 'transformation_label': 'Binarizer'}], 'feature': 'beneficiary_cost_share_lis', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '254', 'mean': '3132957.81', 'stddev': '5067461.93', 'min': '0.0', 'max': '2.498282348E7', 'missing': '246'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'beneficiary_cost_share_li...'}, {'feature_label': 'beneficiary_cost_share_lis', 'threshold': 3132957.16, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop(
            'beneficiary_cost_share_lis')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'beneficiary_cost_share_no_lis', 'threshold': 44993208.766, 'transformation_label': 'Binarizer'}], 'feature': 'beneficiary_cost_share_no_lis', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '254', 'mean': '44993209.05', 'stddev': '70057281.46', 'min': '0.0', 'max': '3.7454264254E8', 'missing': '246'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'beneficiary_cost_share_no...'}, {'feature_label': 'beneficiary_cost_share_no_lis', 'threshold': 44993208.766, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop(
            'beneficiary_cost_share_no_lis')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run medicareprescriptiondrugspendingHooks.ipynb
try:
	#sourcePreExecutionHook()

	medicarerxspending = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/health-medicare-drug-spending (1).csv', 'filename': 'health-medicare-drug-spending (1).csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run medicareprescriptiondrugspendingHooks.ipynb
try:
	#transformationPreExecutionHook()

	medicarerxspendingautofe = TransformationMain.run(medicarerxspending,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2012.5", "stddev": "1.62", "min": "2010", "max": "2015", "missing": "0"}, "updatedLabel": "year"}, {"transformationsData": [{"feature_label": "brand_name", "transformation_label": "String Indexer"}], "feature": "brand_name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Abilify", "max": "Tracleer", "missing": "0", "distinct": "92"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "brand_name"}, {"transformationsData": [{"feature_label": "generic_name", "transformation_label": "String Indexer"}], "feature": "generic_name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Abatacept; Abatacept/Maltose", "max": "Treprostinil Sodium", "missing": "0", "distinct": "89"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "generic_name"}, {"transformationsData": [{"feature_label": "coverage_type", "transformation_label": "String Indexer"}], "feature": "coverage_type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Part B", "max": "Part D", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "coverage_type"}, {"transformationsData": [{"feature_label": "total_spending", "threshold": 448085386.374, "transformation_label": "Binarizer"}], "feature": "total_spending", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "469982921.3", "stddev": "685975898.26", "min": "0.0", "max": "7.0306334858E9", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "total_spending"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "beneficiary_count", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "245752.68", "stddev": "466065.51", "min": "0", "max": "5698568", "missing": "0"}, "updatedLabel": "beneficiary_count"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "unit_count", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "54229117.19", "stddev": "102945176.13", "min": "0", "max": "492962520", "missing": "0"}, "updatedLabel": "unit_count"}, {"transformationsData": [{"feature_label": "beneficiary_cost_share", "threshold": 53223939.044, "transformation_label": "Binarizer"}], "feature": "beneficiary_cost_share", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "53223939.48", "stddev": "76494193.61", "min": "0.0", "max": "3.9106178122E8", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "beneficiary_cost_share"}, {"transformationsData": [{"feature_label": "total_annual_spending_per_user", "threshold": 15359.996, "transformation_label": "Binarizer"}], "feature": "total_annual_spending_per_user", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "15360.47", "stddev": "25215.13", "min": "0.0", "max": "162370.93", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "total_annual_spending_per..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "average_cost_per_unit", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "582.97", "stddev": "3397.72", "min": "0.0", "max": "35205.98", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "average_cost_per_unit"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "average_annual_beneficiary_cost_share", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "233", "mean": "3214.92", "stddev": "4998.57", "min": "0.0", "max": "29287.8", "missing": "267"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "average_annual_beneficiar..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "claim_count", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1079153.5", "stddev": "1906005.1", "min": "0", "max": "9073952", "missing": "0"}, "updatedLabel": "claim_count"}, {"transformationsData": [{"feature_label": "beneficiary_count_lis", "transformation_label": "String Indexer"}], "feature": "beneficiary_count_lis", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "254", "mean": "128.85", "stddev": "228.59", "min": "0", "max": "99,200", "missing": "246", "distinct": "238"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "beneficiary_count_lis"}, {"transformationsData": [{"feature_label": "beneficiary_count_no_lis", "transformation_label": "String Indexer"}], "feature": "beneficiary_count_no_lis", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "254", "mean": "219.1", "stddev": "296.05", "min": "0", "max": "971,937", "missing": "246", "distinct": "238"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "beneficiary_count_no_lis"}, {"transformationsData": [{"feature_label": "beneficiary_cost_share_lis", "threshold": 3132957.16, "transformation_label": "Binarizer"}], "feature": "beneficiary_cost_share_lis", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "254", "mean": "3132957.81", "stddev": "5067461.93", "min": "0.0", "max": "2.498282348E7", "missing": "246"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "beneficiary_cost_share_li..."}, {"transformationsData": [{"feature_label": "beneficiary_cost_share_no_lis", "threshold": 44993208.766, "transformation_label": "Binarizer"}], "feature": "beneficiary_cost_share_no_lis", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "254", "mean": "44993209.05", "stddev": "70057281.46", "min": "0.0", "max": "3.7454264254E8", "missing": "246"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "beneficiary_cost_share_no..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "average_beneficiary_cost_share_lis", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "254", "mean": "36.56", "stddev": "43.99", "min": "0.0", "max": "216.0", "missing": "246"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "average_beneficiary_cost_..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "average_beneficiary_cost_share_no_lis", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "254", "mean": "1184.61", "stddev": "1605.68", "min": "0.0", "max": "8007.22", "missing": "246"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "average_beneficiary_cost_..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "serialid", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "250.5", "stddev": "144.48", "min": "1", "max": "500", "missing": "0"}, "updatedLabel": "serialid"}]}))

	#transformationPostExecutionHook(medicarerxspendingautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run medicareprescriptiondrugspendingHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(medicarerxspendingautofe, ["year", "unit_count", "average_cost_per_unit", "average_annual_beneficiary_cost_share", "claim_count", "average_beneficiary_cost_share_lis", "average_beneficiary_cost_share_no_lis", "serialid", "brand_name_stringindexer", "generic_name_stringindexer", "coverage_type_stringindexer", "total_spending_binarizer", "beneficiary_cost_share_binarizer", "total_annual_spending_per_user_binarizer", "beneficiary_count_lis_stringindexer", "beneficiary_count_no_lis_stringindexer", "beneficiary_cost_share_lis_binarizer", "beneficiary_cost_share_no_lis_binarizer"], "beneficiary_count")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

